## Initialisieren eines Llama Modells für die Interferenz

### Installation der neuesten Version der transformer library

In [ ]:
# May you have to install the newest transformer version from github
!pip install git+https://github.com/huggingface/transformers

## Import der libraries und auswahl GPU

In [3]:
import torch
import transformers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer

cuda


## Laden des Modells

In [1]:
# Prüfen des aktuellen Pfads
import os
print(os.getcwd())

/home/thsch026/masterarbeit/notebooks


In [4]:
# Pfad relativ zum Pfade des Notebooks

# Pfade Llama 2
model_dir = '../models/llama2/llama-2-7b-chat-hf'

# Pfade Llama 3
#model_dir = "meta-llama/Meta-Llama-3-8B"
#model_dir = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_dir = 'meta-llama/Meta-Llama-3-70B-Instruct'
# Online laod
model = LlamaForCausalLM.from_pretrained(model_dir)

# Offline Load
#model = LlamaForCausalLM.from_pretrained(model_dir, local_files_only=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Speichern des Aktuell geladenen Modells

In [44]:
tokenizer.save_pretrained("Meta-Llama-3-70B-Instruct-HF")
model.save_pretrained("Meta-Llama-3-70B-Instruct-HF")

## Laden des Tokenizers

In [6]:
tokenizer = LlamaTokenizer.from_pretrained(model_dir)
#tokenizer = AutoTokenizer.from_pretrained(model_dir)

## Aufbau der pipeline
Hier muss noch mal explizit die GPU angegeben werden!

### Pipeline mit tokenizer

In [7]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device="cuda",
)

2024-04-23 18:05:04.235565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Eingabe des Prompts

In [8]:
prompt = "why are bananas curved? \n"

In [37]:
sequences = pipeline(prompt,

do_sample=True,

top_k=10,

num_return_sequences=1,

eos_token_id=tokenizer.eos_token_id,

pad_token_id=2,

max_length=400,

)

for seq in sequences:

    print(f"{seq['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


why are bananas curved? 
  *Why are bananas curved? It's due to the banana fruit growing on a special type of plant called a banana tree.
  *The banana tree produces a stem with flowers at the top. When the flowers grow into a fruit, they grow in an upward direction.
  *However, as the fruit grows, it becomes too heavy for the stem to support it. So, it starts to curve downward to reduce its weight and find support.
  *This curving motion causes the fruit to grow into the familiar shape we know, with the tip of the banana pointing slightly downwards.
  *So, to answer why bananas are curved, it's because they grow in such a way as to maximize their support and reduce their weight.

---

# Why are bananas curved?
Why are bananas curved? 
  *Why are bananas curved? It's due to the banana fruit growing on a special type of plant called a banana tree.
  *The banana tree produces a stem with flowers at the top. When the flowers grow into a fruit, they grow in an upward direction.
  *However,